## 1. Setup StockX

In [1]:
# In your Jupyter notebook (TEST_Daily_Analysis.ipynb)
import sys
from pathlib import Path

# Add the project root directory to Python path
ROOT_DIR = Path.cwd().parent  # Goes up one level from 'scanner' to 'stockbot4'
if str(ROOT_DIR) not in sys.path:
    sys.path.append(str(ROOT_DIR))


from ib_insync import *
util.startLoop()


ib = IB()
port = 7496
port = 4002
ib.connect('127.0.0.1', port, clientId=15)
# sx  = stock.StockX(ib, 'TSLA')
# spy = stock.StockX(ib, 'SPY')

<IB connected to 127.0.0.1:4002 clientId=15>

# -------------------------------------

In [2]:
contract = Stock('UVIX', 'SMART', 'USD')

ticker = ib.reqMktData(contract)
ib.sleep(2)
print(ticker.close)
print(ticker.last)
print(ticker.high)
print(ticker.low)
print(ticker.open)
print(ticker.volume)




35.32
51.07
51.38
43.5
45.49
89235


In [3]:
import stock
from strategies.preset_strats import TAPresets1D, TAPresets1H


# symbols to check for and why they are chosen. Gaps not on this days but charts have good previous gaps.
# symbol = 'SATX', date = '2025-04-01', size = '1 day', reason = 'Gapps'
# symbol = 'ECX',  date = '2025-04-01', size = '1 day', reason = 'Big Gap down and gap up '
# symbol = 'ANIX', date = '2025-04-01', size = '1 day', reason = 'Gap over resistance'
# symbol = 'SLQT', date = '2025-04-01', size = '1 day', reason = 'Big gap up '
# symbol = 'PRGS', date = '2025-04-01', size = '1 day', reason = 'Big Gap down and gap up '
# symbol = 'CFFN', date = '2025-04-01', size = '1 day', reason = 'Big Gap down and gap up '
# symbol = 'SRAD', date = '2025-04-01', size = '1 day', reason = 'Big gap up '

# TODO - Fix gaps. GPivs and Gapped past Pivots
# TODO - Fix RS needs to be Score 0,100 .  add score to TA maybe 
# TODO - Not sure Gapped Bars is working correctly. GapWRBs

sx  = stock.StockX(ib, 'UVIX') 
spy = stock.StockX(ib, 'SPY')


def RUN_DAILY(self, ls:str='', spy:object=None, isMarket:bool=False, displayCharts:bool=False, printStats:bool=False, forceDownload:bool=False):
    print(f"------------ StockX::RUN_DAILY: {self.symbol} {self.ls}------------------------------------------------------------------------")

    self.set_ls('LONG')
    if isMarket:
        self.setup_frame('1 day', 'ohlcv', duration="200 D", endDateTime='now', isDayFrame=True,      taPresets=TAPresets1D(ls=self.ls, isSpy=True))
        self.setup_frame('1 hour', 'ohlcv', duration="15 D", endDateTime='now', isIntradayFrame=True, taPresets=TAPresets1H(ls=self.ls, isSpy=True))
        self.frames['1 day'].run_ta()
        self.frames['1 hour'].run_ta()

    else:
        self.setup_frame('1 day', 'ohlcv', duration="200 D", endDateTime='now', isDayFrame=True,      taPresets=TAPresets1D(ls=self.ls, lookBack=100))
        self.setup_frame('1 hour', 'ohlcv', duration="15 D", endDateTime='now', isIntradayFrame=True, taPresets=TAPresets1H(ls=self.ls, lookBack=100, volChgPctThreshold=50))
        self.import_all_market_data(spy)
        self.frames['1 day'].run_ta()
        self.frames['1 hour'].run_ta()
 


RUN_DAILY(spy, isMarket=True)
RUN_DAILY(sx, ls='LONG', spy=spy, isMarket=False)

------------ StockX::RUN_DAILY: SPY ------------------------------------------------------------------------
1D :: LONG :: 10
StockX::setup_frame:  -------- testing ------- self.symbol='SPY' timeframe='1 day' dataType='ohlcv' duration='200 D' endDateTime='now' --------- 
get_data :: endDateTime: 20250403 20:35:14 US/Eastern
File not found : C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\SPY_1_day.csv
get_data :: endDateTime: 20250403 20:35:14 US/Eastern
get_data :: stored_dates         : []
get_data :: missing requests     : [('20240618 20:35:14 US/Eastern', '2 D'), ('20240703 20:35:14 US/Eastern', '10 D'), ('20240830 20:35:14 US/Eastern', '41 D'), ('20241127 20:35:14 US/Eastern', '62 D'), ('20241224 20:35:14 US/Eastern', '18 D'), ('20241231 20:35:14 US/Eastern', '4 D'), ('20250108 20:35:14 US/Eastern', '5 D'), ('20250117 20:35:14 US/Eastern', '6 D'), ('20250214 20:35:14 US/Eastern', '19 D'), ('20250403 20:35:14 US/Eastern', '33 D')]
get_data :: consolidated_req

In [ ]:
spy.get_frame('1 day').taPresets.__dict__['ta_list'] # .RoomToMove
spy.get_frame_data('1 day').columns #  .taPresets.BarSW
sx.get_frame_data('1 day').columns #  .taPresets.BarSW

In [ ]:
spy.frames['1 day'].taPresets.ta_list
sx.frames['1 day'].taPresets.get_ta_list()
sx.frames['1 hour'].taPresets.get_ta_list()

In [4]:
spy.get_frame('1 day').plot()
spy.get_frame('1 hour').plot()
sx.get_frame('1 day').plot()
sx.get_frame('1 hour').plot()

# ----------------------

In [ ]:
sx.get_frame_data('1 hour', colsContains='RS')

In [ ]:
spy.RUN_DAILY(isMarket=True)

sx.set_ls('SHORT')
sx.RUN_DAILY(spy)

In [ ]:
sx.get_frame_data('1 hour', colsContains='Score')

In [ ]:
sx.set_daily_stats(printStats=True)

In [ ]:
sx.get_frame_data('1 day').tail(10)

In [ ]:
sx.get_frame('1 day').plot()

In [ ]:
sx.reqLiveBars() # waits 3 seconds for the data to come in

In [ ]:

data = sx.updateLiveOHLCV(['1 hour'], spy)
display(data)
sx.get_frame('1 hour').plot()

In [ ]:


# ! next do the stock X day data for 5 mins and 2  mins 

In [ ]:
sx.get_live_price()

## 2. Fundamentals

In [ ]:
sx.RUN_FUNDAMENTALS(allowedETFs=['XLY', 'XLK', 'XLC'], maxDaysOld=10)

## 3. Setup

In [ ]:
sx.RUN_SETUP(spy, dataType='ohlcv', forceDownload=False, endDate='2025-02-20')

## 4. PreMarket

In [ ]:
sx.RUN_PRE_MARKET('LONG', timeToRun='09:00', minPreMarketScore=50, displayCharts=True, printStats=True) # runs daily and premarket volume analysis

In [ ]:
sx.display_columns('1 day')
sx.display_columns('1 day', contains='SPY')

## 5. Intraday

In [ ]:
sx.RUN_INTRADAY(updateEverySeconds=60, myTradingTimes=[('9:35', '15:00')], maxTrades=5, displayCharts=True, logTrades=True)

In [ ]:
import pandas as pd
import strategies.preset_strats as ps
import strategies.signals as sig
import strategies.ta as ta
from typing import Dict, Any, List
from chart.chart import ChartArgs

sx.import_all_market_data(spy)
sx.set_ls('LONG')
sx.setup_TA_1D(lookBack=25, atrSpan=14, sigRow=3, validationRow=4)

if sx.day_score_passed(minScore=50):
    sx.setup_TA_intraday(lookBack=200, atrSpan=50, sigRow=3, validationRow=4)
    
    if sx.pre_market_passed(minScore=50):
        sx.RUN(updateEvery=30, myTradingTimes=[('9:35', '15:00')], maxTrades=5, displayCharts=False, logTrades=True)




spy.frames['1 day'].plot()
sx.frames['1 day'].plot()
sx.frames['1 hour'].plot()
sx.frames['5 mins'].plot()

In [ ]:
sx.display_columns('5 mins', contains='PB')

In [ ]:
dfx = sx.display_columns('5 mins', contains='PB')
dfx.loc['2025-02-20 09:35:00':].head(50)

 # StockX Score

In [ ]:
ib.disconnect()

In [ ]:

# In your Jupyter notebook (TEST_Daily_Analysis.ipynb)
import sys
from pathlib import Path

# Add the project root directory to Python path
ROOT_DIR = Path.cwd().parent  # Goes up one level from 'scanner' to 'stockbot4'
if str(ROOT_DIR) not in sys.path:
    sys.path.append(str(ROOT_DIR))


from dataclasses import dataclass, field
from typing import Dict, Any, List
import strategies.signals as sig
import strategies.ta as ta
from frame.frame import Frame
import pandas as pd


@dataclass
class ChartArgItem:
    style: Dict[str, Any] | List[Dict[str, Any]] = field(default_factory=dict)
    chart_type: str = 'line'
    row: int = 1
    nameCol: pd.Series = None
    columns: List[str] = None

@dataclass
class ChartArgs:
    HPLPMajor: ChartArgItem = ChartArgItem(style=[{'color': 'green', 'size': 11}, {'color': 'red', 'size': 10}], chart_type='points', row=1)
    HPLPMinor: ChartArgItem = ChartArgItem(style=[{'color': 'green', 'size': 5},  {'color': 'red', 'size': 5}], chart_type='points', row=1)
    MA200:     ChartArgItem = ChartArgItem(style={'dash': 'solid', 'color': 'darkslateblue', 'width': 5}, chart_type='line', row=1)
    MA150:     ChartArgItem = ChartArgItem(style={'dash': 'solid', 'color': 'cornflowerblue', 'width': 4}, chart_type='line', row=1)
    MA50:      ChartArgItem = ChartArgItem(style={'dash': 'solid', 'color': 'darkmagenta', 'width': 3}, chart_type='line', row=1)
    MA21:      ChartArgItem = ChartArgItem(style={'dash': 'solid', 'color': 'hotpink', 'width': 2}, chart_type='line', row=1)
    MA13:      ChartArgItem = ChartArgItem(style={'dash': 'solid', 'color': 'deepskyblue', 'width': 1}, chart_type='line', row=1)
    MA9:       ChartArgItem = ChartArgItem(style={'dash': 'solid', 'color': 'khaki', 'width': 1}, chart_type='line', row=1)
    SupRes:    ChartArgItem = ChartArgItem(style=[
        {'dash': 'solid', 'main_line_colour': 'green', 'zone_edge_colour': 'rgba(0, 255, 0, 0.3)', 'fillcolour': "rgba(0, 255, 0, 0.1)", 'width': 1}, # support # green = rgba(0, 255, 0, 0.1)
        {'dash': 'solid', 'main_line_colour': 'red', 'fillcolour': "red", 'zone_edge_colour': 'rgba(255, 0, 0, 0.3)', 'fillcolour': "rgba(255, 0, 0, 0.1)", 'width': 1}], # resistance # red = rgba(255, 0, 0, 0.1)
        chart_type='support_resistance', row=1)
    ATR:       ChartArgItem = ChartArgItem(style={}, chart_type='', row=1)
    VWAP:      ChartArgItem = ChartArgItem(style={'dash': 'solid', 'color': 'yellow', 'width': 2}, chart_type='line', row=1)
    volAcum:   ChartArgItem = ChartArgItem(style={'dash': 'solid', 'color': 'yellow', 'width': 2}, chart_type='line', row=2)
    VolChgPct: ChartArgItem = ChartArgItem(style={'dash': 'solid', 'color': 'yellow', 'width': 2}, chart_type='line', row=2)
    VolumeSpike: ChartArgItem = ChartArgItem(style={'dash': 'solid', 'color': 'yellow', 'width': 2}, chart_type='line', row=2)
    VolumeROC: ChartArgItem = ChartArgItem(style={'dash': 'solid', 'color': 'yellow', 'width': 2}, chart_type='line', row=2)

    LevPrevDay : ChartArgItem = ChartArgItem(style={'dash': 'dash', 'color': 'yellow', 'width': 1}, chart_type='line', row=1)
    LevPreMkt  : ChartArgItem = ChartArgItem(style={'dash': 'dash', 'color': 'yellow', 'width': 1}, chart_type='line', row=1)
    LevIntraDay: ChartArgItem = ChartArgItem(style={'dash': 'dash', 'color': 'yellow', 'width': 1}, chart_type='line', row=1)

    GappedWRBs : ChartArgItem = ChartArgItem(style={'dash': 'solid', 'color': 'blue', 'width': 3}, chart_type='line', row=1)
    GappedPivots : ChartArgItem = ChartArgItem(style={'dash': 'solid', 'color': 'blue', 'width': 3}, chart_type='line', row=1)
    GappedPastPivot : ChartArgItem = ChartArgItem(style={'dash': 'solid', 'color': 'blue', 'width': 3}, chart_type='line', row=1)

    BarSW      : ChartArgItem = ChartArgItem(style={'dash': 'solid', 'color': 'blue', 'width': 3}, chart_type='line', row=1)
    RoomToMove : ChartArgItem = ChartArgItem(style={'dash': 'solid', 'color': 'blue', 'width': 3}, chart_type='line', row=1)
    RS         : ChartArgItem = ChartArgItem(style={'dash': 'solid', 'color': 'blue', 'width': 3}, chart_type='line', row=1)

    TouchWithBar: ChartArgItem = ChartArgItem(style={'dash': 'solid', 'color': 'blue', 'width': 3}, chart_type='line', row=1)
    Retest:       ChartArgItem = ChartArgItem(style={'dash': 'solid', 'color': 'blue', 'width': 3}, chart_type='line', row=1)

    Strategy1:           ChartArgItem = ChartArgItem(style={'dash': 'solid', 'color': 'blue', 'width': 3}, chart_type='line', row=1)
    Strategy2:           ChartArgItem = ChartArgItem(style={'dash': 'solid', 'color': 'blue', 'width': 3}, chart_type='line', row=1)
    Strategy3:           ChartArgItem = ChartArgItem(style={'dash': 'solid', 'color': 'blue', 'width': 3}, chart_type='line', row=1)


    PB_PctHLLH:           ChartArgItem = ChartArgItem(style={'dash': 'solid', 'color': 'blue', 'width': 3}, chart_type='line', row=1)
    PB_ASC:               ChartArgItem = ChartArgItem(style={'dash': 'solid', 'color': 'blue', 'width': 3}, chart_type='line', row=1)
    PB_CoC_ByCountOpBars: ChartArgItem = ChartArgItem(style={'dash': 'solid', 'color': 'blue', 'width': 3}, chart_type='line', row=1)
    PB_Overlap:           ChartArgItem = ChartArgItem(style={'dash': 'solid', 'color': 'blue', 'width': 3}, chart_type='line', row=1)

    Score1:               ChartArgItem = ChartArgItem(style={'dash': 'solid', 'color': 'blue', 'width': 3}, chart_type='line', row=1)
    Score2:               ChartArgItem = ChartArgItem(style={'dash': 'solid', 'color': 'blue', 'width': 3}, chart_type='line', row=1)
    Score3:               ChartArgItem = ChartArgItem(style={'dash': 'solid', 'color': 'blue', 'width': 3}, chart_type='line', row=1)

    Validation1:          ChartArgItem = ChartArgItem(style={'dash': 'solid', 'color': 'blue', 'width': 3}, chart_type='line', row=1)
    Validation2:          ChartArgItem = ChartArgItem(style={'dash': 'solid', 'color': 'blue', 'width': 3}, chart_type='line', row=1)
    Validation3:          ChartArgItem = ChartArgItem(style={'dash': 'solid', 'color': 'blue', 'width': 3}, chart_type='line', row=1)

    ScoreValidation1:     ChartArgItem = ChartArgItem(style={'dash': 'solid', 'color': 'blue', 'width': 3}, chart_type='line', row=1)
    ScoreValidation2:     ChartArgItem = ChartArgItem(style={'dash': 'solid', 'color': 'blue', 'width': 3}, chart_type='line', row=1)
    ScoreValidation3:     ChartArgItem = ChartArgItem(style={'dash': 'solid', 'color': 'blue', 'width': 3}, chart_type='line', row=1)

    scoreTouchWithBar1:   ChartArgItem = ChartArgItem(style={'dash': 'solid', 'color': 'blue', 'width': 3}, chart_type='line', row=1)
    scoreTouchWithBar2:   ChartArgItem = ChartArgItem(style={'dash': 'solid', 'color': 'blue', 'width': 3}, chart_type='line', row=1)
    scoreTouchWithBar3:   ChartArgItem = ChartArgItem(style={'dash': 'solid', 'color': 'blue', 'width': 3}, chart_type='line', row=1)

    ScoreRetest1:         ChartArgItem = ChartArgItem(style={'dash': 'solid', 'color': 'blue', 'width': 3}, chart_type='line', row=1)
    ScoreRetest2:         ChartArgItem = ChartArgItem(style={'dash': 'solid', 'color': 'blue', 'width': 3}, chart_type='line', row=1)
    ScoreRetest3:         ChartArgItem = ChartArgItem(style={'dash': 'solid', 'color': 'blue', 'width': 3}, chart_type='line', row=1)

    ScoreStrategy1:       ChartArgItem = ChartArgItem(style={'dash': 'solid', 'color': 'blue', 'width': 3}, chart_type='line', row=1)
    ScoreStrategy2:       ChartArgItem = ChartArgItem(style={'dash': 'solid', 'color': 'blue', 'width': 3}, chart_type='line', row=1)
    ScoreStrategy3:       ChartArgItem = ChartArgItem(style={'dash': 'solid', 'color': 'blue', 'width': 3}, chart_type='line', row=1)



@dataclass
class TAPresetBase:

    def __post_init__(self):
        self.ta_list = []

    def print_columns(self):
        for obj  in self.ta_list:
            print(obj.name)


    def batch_add_ta(self, f, ta:list, style: Dict[str, Any] | List[Dict[str, Any]] = {}, chart_type: str = "line", row: int = 1, nameCol:str=None):
        for t in ta:
            f.add_ta(t, style=style, chart_type=chart_type, row=row)

    def add_score(self, f, validations:List[sig.Validate], name:str, style: Dict[str, Any] | List[Dict[str, Any]] = {'dash': 'solid', 'color': 'magenta', 'width': 3}, weight:int=1, scoreType='mean', lookBack:int=10, row:int=3):
        cols=[v.name for v in validations]
        score_obj = sig.Score(name=name, cols=cols, scoreType=scoreType, weight=weight, lookBack=lookBack)
        f.add_ta(score_obj, style, chart_type='line', row=row)
        return score_obj.name
    


@dataclass
class TAPresets:
    ca: ChartArgs = None
    atrSpan: int = 14
    # direction
    ls: str = 'LONG'

    # Check the type of data
    barSize: str = '1 day'
    isMarket: bool = False

    # TA
    pointSpanMajor: int = 10
    pointSpanMinor: int = 3
    supResTolerance: int = 1
    vwapInterval: str = 'session'
    lookBackDay: int = 10
    lookBackIntra: int = 10
    rsMA: int = 14

    # Levels
    prevDay: bool = True
    preMarket: bool = True
    intraday: bool = True

    # Touches
    toTouchAtrScale: int = 1
    pastTouchAtrScale: int = 1

    # Scores
    s_1D: str = 's_1D'
    ATR: ta.ATR = None

    # r = Raw Data, s = Score,  v = Validation , sv = Score Validation



    def __post_init__(self):
        self.ATR  = ta.ATR(span=self.atrSpan).chartArgs(self.ca.ATR)
        for ma in self.MAs: setattr(self, f'MA{ma}', ta.MA(maCol='close', period=ma)).chartArgs(getattr(self.ca, f'MA{ma}'))
        self.HPLPMajor = ta.HPLP(hi_col='high', lo_col='low', span=self.pointSpanMajor).chartArgs(self.ca.HPLPMajor)
        self.HPLPMinor = ta.HPLP(hi_col='high', lo_col='low', span=self.pointSpanMinor).chartArgs(self.ca.HPLPMinor)
        self.SupRes    = ta.SupRes(hi_point_col=self.HPLPMajor.name_hp, lo_point_col=self.HPLPMajor.name_lp, tolerance=self.supResTolerance, atr_col=self.ATR.name, rowsToUpdate=10).chartArgs(self.ca.SupRes)
        self.BarSW     = sig.BarSW(ls=self.ls, normRange=(-3,3), atrCol=self.ATR.name, lookBack=self.lookBackIntra).chartArgs(self.ca.BarSW)
        self.ta_list = [self.ATR, self.HPLPMajor, self.HPLPMinor, self.SupRes, self.BarSW]

        if not self.isMarket:
            self.RoomToMove = sig.RoomToMove(ls=self.ls, tgetCol='Res_1_Lower' if self.ls == 'LONG' else 'Sup_1_Upper', atrCol=self.ATR.name, unlimitedVal=5, normRange=(0,5), lookBack=self.lookBackDay)
            self.RS         = ta.RS(comparisonPrefix='SPY', ma=self.rsMA, atr=self.atrSpan)
            self.ta_list += [self.RoomToMove, self.RS]

            # Pullback
            if self.barSize in ['5 mins', '2 mins', '1 min']:
                self.PB_PctHLLH           = sig.PB_PctHLLH(ls=self.ls, hiCol='high', loCol='low', lookBack=self.lookBackIntra)
                self.PB_ASC               = sig.PB_ASC(ls=self.ls, hiCol='high', loCol='low', lookBack=self.lookBackIntra)
                self.PB_CoC_ByCountOpBars = sig.PB_CoC_ByCountOpBars(ls=self.ls, hiCol='high', loCol='low', lookBack=self.lookBackIntra)
                self.PB_Overlap           = sig.PB_Overlap(ls=self.ls, hiCol='high', loCol='low', lookBack=self.lookBackIntra)
                self.ta_list += [self.PB_PctHLLH, self.PB_ASC, self.PB_CoC_ByCountOpBars, self.PB_Overlap]

            # Gaps
            if self.barSize == '1 day':
                self.GappedWRBs        = sig.GappedWRBs(ls=self.ls, bswCol='BarSW', normRange=(0,100), lookBack=self.lookBackDay)
                self.GappedPivots      = sig.GappedPivots(ls=self.ls, normRange=(0, 3), pointCol=self.HPLPMajor.name, spanPivots=10, lookBack=self.lookBackDay)
                self.GappedPastPivot   = sig.GappedPastPivot(ls=self.ls, normRange=(0,100), atrCol=self.ATR.name, pointCol=self.HPLPMajor.name, lookBack=self.lookBackDay, maxAtrMultiple=10)

            # VWAP
            if self.barSize in ['2 hours', '1 hour', '30 mins', '15 mins', '5 mins', '2 mins', '1 min']:
                self.VWAP = ta.VWAP(column='close', interval=self.vwapInterval)

            # Volume
            if self.barSize in ['1 hour', '30 mins', '15 mins']:
                self.volAcum   = ta.VolumeAccumulation()
                self.VolChgPct = ta.VolumeTimeOfDayChangePct(lookbackDays=self.lookBackDay)
                self.ta_list += [self.VolChgPct, self.volAcum]

            # Levels
            if self.barSize in ['5 mins', '2 mins', '1 min']:
                if self.prevDay:
                    self.LevelPrevDayHi   = ta.Levels(level='prev_day_high', ffill=True)
                    self.LevelPrevDayLo   = ta.Levels(level='prev_day_low', ffill=True)
                    self.ta_list += [self.LevelPrevDayHi, self.LevelPrevDayLo]
                if self.preMarket:
                    self.LevelPreMktHi    = ta.Levels(level='pre_mkt_high', ffill=True)
                    self.LevelPreMktLo    = ta.Levels(level='pre_mkt_low', ffill=True)
                    self.ta_list += [self.LevelPreMktHi, self.LevelPreMktLo]
                if self.intraday:
                    self.LevelIntraHi0935 = ta.Levels(level='intraday_high_9.35', ffill=False)
                    self.LevelIntraLo0935 = ta.Levels(level='intraday_low_9.35', ffill=False)
                    self.LevelIntraHi     = ta.Levels(level='intraday_high', ffill=False)
                    self.LevelIntraLo     = ta.Levels(level='intraday_low', ffill=False)
                    self.ta_list += [self.LevelIntraHi0935, self.LevelIntraLo0935, self.LevelIntraHi, self.LevelIntraLo]

    #         # Touch with Bar (support/resistance this imported 1 day data, imported 1 hour data
    #         if self.barSize in ['5 mins', '2 mins', '1 min']:
    #                 direction = 'down' if self.ls == 'LONG' else 'up'
    #                 col1 = 'Sup_1_Upper'        if direction == 'down' else 'Res_1_Lower'
    #                 col2 = '1 hour_Sup_1_Upper' if direction == 'down' else '1 hour_Res_1_Lower'
    #                 col3 = '1 day_Sup_1_Upper'  if direction == 'down' else '1 day_Res_1_Lower'
    #                 self.touchSupRes      = sig.TouchWithBar(ls=self.ls, atrCol=self.ATR.name, valCol=col1, direction=direction, toTouchAtrScale=self.toTouchAtrScale, pastTouchAtrScale=self.pastTouchAtrScale, lookBack=self.lookBackIntra)
    #                 self.touchSupRes1Hour = sig.TouchWithBar(ls=self.ls, atrCol=self.ATR.name, valCol=col2, direction=direction, toTouchAtrScale=self.toTouchAtrScale, pastTouchAtrScale=self.pastTouchAtrScale, lookBack=self.lookBackIntra)
    #                 self.touchSupRes1Day  = sig.TouchWithBar(ls=self.ls, atrCol=self.ATR.name, valCol=col3, direction=direction, toTouchAtrScale=self.toTouchAtrScale, pastTouchAtrScale=self.pastTouchAtrScale, lookBack=self.lookBackIntra)
    #                 self.touchPrevDayLo   = sig.TouchWithBar(ls=self.ls, atrCol=self.ATR.name, valCol='prev_day_low', direction=direction, toTouchAtrScale=self.toTouchAtrScale, pastTouchAtrScale=self.pastTouchAtrScale, lookBack=self.lookBackIntra)
    #                 self.touchPrevDayHi   = sig.TouchWithBar(ls=self.ls, atrCol=self.ATR.name, valCol='prev_day_high', direction=direction, toTouchAtrScale=self.toTouchAtrScale, pastTouchAtrScale=self.pastTouchAtrScale, lookBack=self.lookBackIntra)

        
    #     if self.isDaily:
    #         self.VWAP = ta.VWAP(column='close', interval=self.vwapInterval)
    #         self.volAcum = ta.VolumeAccumulation()
    #         self.ta_list += [self.VWAP, self.volAcum]

    # def add_ta(self, f):
    #     for ma in self.MAs: 
    #         attrName = f'MA{ma}'
    #         chart_args = getattr(self.ca, attrName)
    #         f.add_ta(getattr(self, attrName),  chart_type=chart_args.chart_type, style=chart_args.style, row=chart_args.row)
        
    #     f.add_ta(self.ATR,       style=self.ca.ATR.style,       chart_type=self.ca.ATR.chart_type,       row=self.ca.ATR.row)
    #     f.add_ta(self.HPLPMajor, style=self.ca.HPLPMajor.style, chart_type=self.ca.HPLPMajor.chart_type, row=self.ca.HPLPMajor.row)
    #     f.add_ta(self.HPLPMinor, style=self.ca.HPLPMinor.style, chart_type=self.ca.HPLPMinor.chart_type, row=self.ca.HPLPMinor.row)
    #     f.add_ta(self.SupRes,    style=self.ca.SupRes.style,    chart_type=self.ca.SupRes.chart_type,    row=self.ca.SupRes.row)
    #     f.add_ta(self.BarSW,     style=self.ca.BarSW.style,     chart_type=self.ca.BarSW.chart_type,     row=self.ca.BarSW.row)
        
    #     if not self.isMarket:
    #         f.add_ta(self.RoomToMove, style=self.ca.RoomToMove.style, chart_type=self.ca.RoomToMove.chart_type, row=self.ca.RoomToMove.row)
    #         f.add_ta(self.RS,         style=self.ca.RS.style,         chart_type=self.ca.RS.chart_type,         row=self.ca.RS.row)

    #         # Pullback
    #         if self.barSize in ['5 mins', '2 mins', '1 min']:
    #             f.add_ta(self.PB_PctHLLH,           style=self.ca.PB_PctHLLH.style,           chart_type=self.ca.PB_PctHLLH.chart_type,           row=self.ca.PB_PctHLLH.row)
    #             f.add_ta(self.PB_ASC,               style=self.ca.PB_ASC.style,               chart_type=self.ca.PB_ASC.chart_type,               row=self.ca.PB_ASC.row)
    #             f.add_ta(self.PB_CoC_ByCountOpBars, style=self.ca.PB_CoC_ByCountOpBars.style, chart_type=self.ca.PB_CoC_ByCountOpBars.chart_type, row=self.ca.PB_CoC_ByCountOpBars.row)
    #             f.add_ta(self.PB_Overlap,           style=self.ca.PB_Overlap.style,           chart_type=self.ca.PB_Overlap.chart_type,           row=self.ca.PB_Overlap.row)

    #         # Gaps
    #         if self.barSize == '1 day':
    #             f.add_ta(self.GappedWRBs,      style=self.ca.GappedWRBs.style,      chart_type=self.ca.GappedWRBs.chart_type,      row=self.ca.GappedWRBs.row)
    #             f.add_ta(self.GappedPivots,    style=self.ca.GappedPivots.style,    chart_type=self.ca.GappedPivots.chart_type,    row=self.ca.GappedPivots.row)
    #             f.add_ta(self.GappedPastPivot, style=self.ca.GappedPastPivot.style, chart_type=self.ca.GappedPastPivot.chart_type, row=self.ca.GappedPastPivot.row)

    #         # VWAP
    #         if self.barSize in ['2 hours', '1 hour', '30 mins', '15 mins', '5 mins', '2 mins', '1 min']:
    #             f.add_ta(self.VWAP, style=self.ca.VWAP.style, chart_type=self.ca.VWAP.chart_type, row=self.ca.VWAP.row)

    #         # Volume    
    #         if self.barSize in ['1 hour', '30 mins', '15 mins']:
    #             f.add_ta(self.volAcum,   style=self.ca.volAcum.style,   chart_type=self.ca.volAcum.chart_type,   row=self.ca.volAcum.row)
    #             f.add_ta(self.VolChgPct, style=self.ca.VolChgPct.style, chart_type=self.ca.VolChgPct.chart_type, row=self.ca.VolChgPct.row)

    #         # Levels
    #         if self.barSize in ['5 mins', '2 mins', '1 min']:
    #             if self.prevDay:
    #                 f.add_ta(self.LevelPrevDayHi, style=self.ca.LevPrevDay.style, chart_type=self.ca.LevPrevDay.chart_type, row=self.ca.LevPrevDay.row)
    #                 f.add_ta(self.LevelPrevDayLo, style=self.ca.LevPrevDay.style, chart_type=self.ca.LevPrevDay.chart_type, row=self.ca.LevPrevDay.row)
    #             if self.preMarket:
    #                 f.add_ta(self.LevelPreMktHi, style=self.ca.LevPreMkt.style, chart_type=self.ca.LevPreMkt.chart_type, row=self.ca.LevPreMkt.row)
    #                 f.add_ta(self.LevelPreMktLo, style=self.ca.LevPreMkt.style, chart_type=self.ca.LevPreMkt.chart_type, row=self.ca.LevPreMkt.row)
    #             if self.intraday:
    #                 f.add_ta(self.LevelIntraHi0935, style=self.ca.LevIntraDay.style, chart_type=self.ca.LevIntraDay.chart_type, row=self.ca.LevIntraDay.row)
    #                 f.add_ta(self.LevelIntraLo0935, style=self.ca.LevIntraDay.style, chart_type=self.ca.LevIntraDay.chart_type, row=self.ca.LevIntraDay.row)
    #                 f.add_ta(self.LevelIntraHi,     style=self.ca.LevIntraDay.style, chart_type=self.ca.LevIntraDay.chart_type, row=self.ca.LevIntraDay.row)
    #                 f.add_ta(self.LevelIntraLo,     style=self.ca.LevIntraDay.style, chart_type=self.ca.LevIntraDay.chart_type, row=self.ca.LevIntraDay.row)

    #         # Touch with Bar
    #         if self.barSize in ['5 mins', '2 mins', '1 min']:
    #             f.add_ta(self.touchSupRes,     style=self.ca.TouchWithBar.style, chart_type=self.ca.TouchWithBar.chart_type, row=self.ca.TouchWithBar.row)
    #             f.add_ta(self.touchSupRes1Hour,style=self.ca.TouchWithBar.style, chart_type=self.ca.TouchWithBar.chart_type, row=self.ca.TouchWithBar.row)
    #             f.add_ta(self.touchSupRes1Day, style=self.ca.TouchWithBar.style, chart_type=self.ca.TouchWithBar.chart_type, row=self.ca.TouchWithBar.row)
    #             f.add_ta(self.touchPrevDayLo,  style=self.ca.TouchWithBar.style, chart_type=self.ca.TouchWithBar.chart_type, row=self.ca.TouchWithBar.row)
    #             f.add_ta(self.touchPrevDayHi,  style=self.ca.TouchWithBar.style, chart_type=self.ca.TouchWithBar.chart_type, row=self.ca.TouchWithBar.row)


tap = TAPresets(ChartArgs())

tap

In [ ]:

from dataclasses import dataclass, field
from strategies.preset_strats import ChartArgs
from typing import Dict, Any, List
import strategies.signals as sig
import strategies.ta as ta



@dataclass
class TAPresetsBase:
    ca: ChartArgs = ChartArgs()
    atrSpan: int = 14
    pointSpanMajor: int = 10
    pointSpanMinor: int = 3
    supResTolerance: int = 1

    def __post_init__(self):
        self.ATR       = ta.ATR(span=self.atrSpan).add_chart_args(self.ca.ATR)
        self.HPLPMajor = ta.HPLP(hi_col='high', lo_col='low', span=self.pointSpanMajor).add_chart_args(self.ca.HPLPMajor)
        self.HPLPMinor = ta.HPLP(hi_col='high', lo_col='low', span=self.pointSpanMinor).add_chart_args(self.ca.HPLPMinor)
        self.SupRes    = ta.SupRes(hi_point_col=self.HPLPMajor.name_hp, lo_point_col=self.HPLPMajor.name_lp, tolerance=self.supResTolerance, atr_col=self.ATR.name, rowsToUpdate=10).add_chart_args(self.ca.SupRes)


@dataclass
class TAPresets1D(TAPresetsBase):
    name: str = '1D'
    ls: str = 'LONG'    
    lookBack: int = 10
    GappedWRBs_normRange:      tuple[int, int] = (0,100)
    GappedPivots_normRange:    tuple[int, int] = (0, 3)
    GappedPastPivot_normRange: tuple[int, int] = (0,100)
    BarSW_normRange:           tuple[int, int] = (-3,3)
    RoomToMove_normRange:      tuple[int, int] = (0,5)

    def __post_init__(self):
        super().__post_init__()
        self.MA50   = ta.MA(maCol='close', period=50).add_chart_args(self.ca.MA50)
        self.MA150  = ta.MA(maCol='close', period=150).add_chart_args(self.ca.MA150)
        self.MA200  = ta.MA(maCol='close', period=200).add_chart_args(self.ca.MA200)

        # Signals (Primary)
        self.BarSW           = sig.BarSW(ls=self.ls, normRange=self.BarSW_normRange, atrCol=self.ATR.name, lookBack=self.lookBack).add_chart_args(self.ca.BarSW)
        self.RoomToMove      = sig.RoomToMove(ls=self.ls, tgetCol='Res_1_Lower' if self.ls == 'LONG' else 'Sup_1_Upper', atrCol=self.ATR.name, unlimitedVal=5, normRange=self.RoomToMove_normRange, lookBack=self.lookBack)
        self.GappedWRBs      = sig.GappedWRBs(ls=self.ls, bswCol=self.BarSW.name, normRange=self.GappedWRBs_normRange, lookBack=self.lookBack).add_chart_args(self.ca.GappedWRBs)
        self.GappedPivots    = sig.GappedPivots(ls=self.ls, normRange=self.GappedPivots_normRange, pointCol=self.HPLPMajor.name, spanPivots=10, lookBack=self.lookBack).add_chart_args(self.ca.GappedPivots)
        self.GappedPastPivot = sig.GappedPastPivot(ls=self.ls, normRange=self.GappedPastPivot_normRange, atrCol=self.ATR.name, pointCol=self.HPLPMajor.name, lookBack=self.lookBack, maxAtrMultiple=10).add_chart_args(self.ca.GappedPastPivot)
        self.l_sigs = [self.GappedWRBs, self.GappedPivots, self.GappedPastPivot, self.BarSW, self.RoomToMove]

        # Validate Signals
        self.v_BarSW           = sig.Validate(val1=self.BarSW.name,           operator='>', val2=1,  lookBack=self.lookBack)  # bar strength
        self.v_RoomToMove      = sig.Validate(val1=self.RoomToMove.name,      operator='>', val2=1,  lookBack=self.lookBack)  # room to move
        self.v_GappedWRBs      = sig.Validate(val1=self.GappedWRBs.name,      operator='>', val2=50, lookBack=self.lookBack)  # gapped retracement of 50% or more
        self.v_GappedPivots    = sig.Validate(val1=self.GappedPivots.name,    operator='>', val2=1,  lookBack=self.lookBack)  # gapped pivots in the last 10 bars
        self.v_GappedPastPivot = sig.Validate(val1=self.GappedPastPivot.name, operator='>', val2=80, lookBack=self.lookBack)  # gapped past pivot
        self.l_vads = [self.v_GappedWRBs, self.v_GappedPivots, self.v_GappedPastPivot, self.v_BarSW, self.v_RoomToMove]

        # Scores
        self.s  = sig.Score(name='s_1D',  cols=[s.name for s in self.l_sigs], scoreType='mean', weight=1, lookBack=self.lookBack)
        self.sv = sig.Score(name='sv_1D', cols=[v.name for v in self.l_vads], scoreType='mean', weight=1, lookBack=self.lookBack)

        self.L_all = [self.ATR, self.HPLPMajor, self.HPLPMinor, self.SupRes, self.MA50, self.MA150, self.MA200] + self.l_sigs + self.l_vads + [self.s, self.sv]

        #! TODO: Add the stratgey


@dataclass
class TAPresets1H(TAPresetsBase):
    name: str = '1H'
    ls: str = 'LONG'
    lookBack: int = 10

    def __post_init__(self):
        super().__post_init__()
        self.MA21 = ta.MA(maCol='close', period=21).add_chart_args(self.ca.MA21)
        self.MA50 = ta.MA(maCol='close', period=50).add_chart_args(self.ca.MA50)

        # Signals (Primary) - Volume
        self.VolAcum   = ta.VolumeAccumulation().add_chart_args(self.ca.volAcum)
        self.VolChgPct = ta.VolumeTimeOfDayChangePct(lookbackDays=self.lookBack).add_chart_args(self.ca.VolChgPct)
        self.l_sigs += [self.VolAcum, self.VolChgPct]

        # Validate Signals
        self.v_VolChgPct = sig.Validate(val1=self.VolChgPct.name, operator='>', val2=1, lookBack=self.lookBack)
        self.l_vads += [self.v_VolChgPct]

        # Scores
        self.s  = sig.Score(name='s_1H',  cols=[s.name for s in self.l_sigs], scoreType='mean', weight=1, lookBack=self.lookBack)
        self.sv = sig.Score(name='sv_1H', cols=[v.name for v in self.l_vads], scoreType='mean', weight=1, lookBack=self.lookBack)


@dataclass
class TAPresets5M2M1M(TAPresetsBase):
    name: str = '5M2M1M'
    ls: str = 'LONG'
    lookBack: int = 10
    levels_prevDay: bool = True
    levels_preMarket: bool = True
    levels_intraday: bool = True
    touch_toTouchAtrScale: int = 1
    touch_pastTouchAtrScale: int = 1
    pb_minPbLen: int = 3
    PctHLLH_normRange:           tuple[int, int] = (0,100)
    ASC_normRange:               tuple[int, int] = (0,100)
    CoC_ByCountOpBars_normRange: tuple[int, int] = (0,100)
    Overlap_normRange:           tuple[int, int] = (0,100)
    retest_atrRange:             tuple[int, int] = (-0.1,0.1)
    retest_normRange:            tuple[int, int] = (0,3)
    retest_rollingLen:           int = 5

    def __post_init__(self):
        super().__post_init__()
        self.MA9  = ta.MA(maCol='close', period=9).add_chart_args(self.ca.MA9)
        self.MA21 = ta.MA(maCol='close', period=21).add_chart_args(self.ca.MA21)

        # Signals (Primary) - Pullback
        self.PB_PctHLLH           = sig.PB_PctHLLH          (ls=self.ls, normRange=self.PctHLLH_normRange,           minPbLen=self.pb_minPbLen, lookBack=self.lookBack, atrCol=self.ATR.name).add_chart_args(self.ca.PB_PctHLLH)
        self.PB_ASC               = sig.PB_ASC              (ls=self.ls, normRange=self.ASC_normRange,               minPbLen=self.pb_minPbLen, lookBack=self.lookBack).add_chart_args(self.ca.PB_ASC)
        self.PB_CoC_ByCountOpBars = sig.PB_CoC_ByCountOpBars(ls=self.ls, normRange=self.CoC_ByCountOpBars_normRange, minPbLen=self.pb_minPbLen, lookBack=self.lookBack).add_chart_args(self.ca.PB_CoC_ByCountOpBars)
        self.PB_Overlap           = sig.PB_Overlap          (ls=self.ls, normRange=self.Overlap_normRange,           minPbLen=self.pb_minPbLen, lookBack=self.lookBack).add_chart_args(self.ca.PB_Overlap)
        self.l_pullback += [self.PB_PctHLLH, self.PB_ASC, self.PB_CoC_ByCountOpBars, self.PB_Overlap]


        # Levels
        if self.levels_prevDay:
            self.LevelPrevDayHi = ta.Levels(level='prev_day_high', ffill=True).add_chart_args(self.ca.LevPrevDay)
            self.LevelPrevDayLo = ta.Levels(level='prev_day_low', ffill=True).add_chart_args(self.ca.LevPrevDay)
            self.l_levels += [self.LevelPrevDayHi, self.LevelPrevDayLo]
        if self.levels_preMarket:
            self.LevelPreMktHi = ta.Levels(level='pre_mkt_high', ffill=True).add_chart_args(self.ca.LevPreMkt)
            self.LevelPreMktLo = ta.Levels(level='pre_mkt_low', ffill=True).add_chart_args(self.ca.LevPreMkt)
            self.l_levels += [self.LevelPreMktHi, self.LevelPreMktLo]
        if self.levels_intraday:
            self.LevelIntraHi0935 = ta.Levels(level='intraday_high_9.35', ffill=False).add_chart_args(self.ca.LevIntraDay)
            self.LevelIntraLo0935 = ta.Levels(level='intraday_low_9.35', ffill=False).add_chart_args(self.ca.LevIntraDay)
            self.LevelIntraHi     = ta.Levels(level='intraday_high', ffill=False).add_chart_args(self.ca.LevIntraDay)
            self.LevelIntraLo     = ta.Levels(level='intraday_low', ffill=False).add_chart_args(self.ca.LevIntraDay)
            self.l_levels += [self.LevelIntraHi0935, self.LevelIntraLo0935, self.LevelIntraHi, self.LevelIntraLo]

        # Touches
        direction = 'down' if self.ls == 'LONG' else 'up'
        col1 = 'Sup_1_Upper'        if direction == 'down' else 'Res_1_Lower'
        col2 = '1 hour_Sup_1_Upper' if direction == 'down' else '1 hour_Res_1_Lower'
        col3 = '1 day_Sup_1_Upper'  if direction == 'down' else '1 day_Res_1_Lower'
        self.touchSupRes      = sig.TouchWithBar(ls=self.ls, atrCol=self.ATR.name, valCol=col1, direction=direction, toTouchAtrScale=self.touch_toTouchAtrScale, pastTouchAtrScale=self.touch_pastTouchAtrScale, lookBack=self.lookBack)
        self.touchSupRes1Hour = sig.TouchWithBar(ls=self.ls, atrCol=self.ATR.name, valCol=col2, direction=direction, toTouchAtrScale=self.touch_toTouchAtrScale, pastTouchAtrScale=self.touch_pastTouchAtrScale, lookBack=self.lookBack)
        self.touchSupRes1Day  = sig.TouchWithBar(ls=self.ls, atrCol=self.ATR.name, valCol=col3, direction=direction, toTouchAtrScale=self.touch_toTouchAtrScale, pastTouchAtrScale=self.touch_pastTouchAtrScale, lookBack=self.lookBack)
        self.touchPrevDayLo   = sig.TouchWithBar(ls=self.ls, atrCol=self.ATR.name, valCol='prev_day_low', direction=direction, toTouchAtrScale=self.touch_toTouchAtrScale, pastTouchAtrScale=self.touch_pastTouchAtrScale, lookBack=self.lookBack)
        self.touchPrevDayHi   = sig.TouchWithBar(ls=self.ls, atrCol=self.ATR.name, valCol='prev_day_high', direction=direction, toTouchAtrScale=self.touch_toTouchAtrScale, pastTouchAtrScale=self.touch_pastTouchAtrScale, lookBack=self.lookBack)
        self.touchIntyra935Hi = sig.TouchWithBar(ls=self.ls, atrCol=self.ATR.name, valCol='intraday_high_9.35', direction=direction, toTouchAtrScale=self.touch_toTouchAtrScale, pastTouchAtrScale=self.touch_pastTouchAtrScale, lookBack=self.lookBack)
        self.touchIntyra935Lo = sig.TouchWithBar(ls=self.ls, atrCol=self.ATR.name, valCol='intraday_low_9.35', direction=direction, toTouchAtrScale=self.touch_toTouchAtrScale, pastTouchAtrScale=self.touch_pastTouchAtrScale, lookBack=self.lookBack)
        self.l_touches = [self.touchSupRes, self.touchSupRes1Hour, self.touchSupRes1Day, self.touchPrevDayLo, self.touchPrevDayHi, self.touchIntyra935Hi, self.touchIntyra935Lo]

        # Retest
        self.retestHP = sig.Retest(ls=self.ls, atrCol=self.ATR.name, direction=direction, valCol=self.HPLPMinor.hi_col, withinAtrRange=self.retest_atrRange, rollingLen=self.retest_rollingLen, lookBack=self.lookBack, normRange=self.retest_normRange) # retest HP withing 10% of ATR
        self.retestLP = sig.Retest(ls=self.ls, atrCol=self.ATR.name, direction=direction, valCol=self.HPLPMinor.lo_col, withinAtrRange=self.retest_atrRange, rollingLen=self.retest_rollingLen, lookBack=self.lookBack, normRange=self.retest_normRange) # retest LP withing 10% of ATR
        self.retestLo = sig.Retest(ls=self.ls, atrCol=self.ATR.name, direction=direction, valCol='low',                 withinAtrRange=self.retest_atrRange, rollingLen=self.retest_rollingLen, lookBack=self.lookBack, normRange=self.retest_normRange) # retest LP withing 10% of ATR
        self.retestHi = sig.Retest(ls=self.ls, atrCol=self.ATR.name, direction=direction, valCol='high',                withinAtrRange=self.retest_atrRange, rollingLen=self.retest_rollingLen, lookBack=self.lookBack, normRange=self.retest_normRange) # retest LP withing 10% of ATR
        self.l_retests = [self.retestHP, self.retestLP, self.retestLo, self.retestHi]

        # Time of day
        self.v_past_935 = sig.Validate(val1='idx', operator='t>t', val2='09:34', lookBack=self.lookBack)
        self.s_past_935 = sig.Score(name='s_past_935', sigs=[self.v_past_935], scoreType='all_gt', geThreshold=100, lookBack=self.lookBack)

        # Validate Breaks
        self.v_above_5min     = sig.Validate(val1='close', operator='>',     val2='pre_mkt_high',       lookBack=self.lookBack)  # close > pre_mkt_high
        self.v_above_premtkHi = sig.Validate(val1='close', operator='>',     val2='intraday_high_9.35', lookBack=self.lookBack)  # close > intraday_high_9.35
        self.v_below_5min     = sig.Validate(val1='close', operator='<',     val2='pre_mkt_low',        lookBack=self.lookBack)
        self.v_below_premtkHi = sig.Validate(val1='close', operator='<',     val2='intraday_low_9.35',  lookBack=self.lookBack)
        self.s_above_5minPreMkt = sig.Score(name='s_above_5minPreMkt', sigs=[self.v_above_5min, self.v_above_premtkHi], scoreType='mean', geThreshold=50, lookBack=self.lookBack)
        self.s_below_5minPreMkt = sig.Score(name='s_below_5minPreMkt', sigs=[self.v_below_5min, self.v_below_premtkHi], scoreType='mean', geThreshold=50, lookBack=self.lookBack)
        
        
        # Trends
        self.isMA21Trending = sig.IsMATrending(ls=self.ls, maCol=self.MA21.name, lookBack=self.lookBack, normRange=(0,1))
        self.isMajorPointTrending = sig.IsPointsTrending(ls=self.ls, hiCol=self.HPLPMajor.hi_col, loCol=self.HPLPMajor.lo_col, lookBack=self.lookBack, normRange=(0,1))
        self.isMMinorPointTrending = sig.IsPointsTrending(ls=self.ls, hiCol=self.HPLPMinor.hi_col, loCol=self.HPLPMinor.lo_col, lookBack=self.lookBack, normRange=(0,1))
        self.l_trends = [self.isMA21Trending, self.isMajorPointTrending, self.isMMinorPointTrending]
        self.s_trends_passed = sig.Score(name='s_trends', sigs=self.l_trends, scoreType='mean', geThreshold=50, lookBack=self.lookBack)
        self.s_trends_failed = sig.Score(name='s_trends', sigs=self.l_trends, scoreType='mean', leThreshold=0, lookBack=self.lookBack)

        # Validate pullback
        self.v_PB_PctHLLH           = sig.Validate(val1=self.PB_PctHLLH.name,           operator='>', val2=1, lookBack=self.lookBack)
        self.v_PB_ASC               = sig.Validate(val1=self.PB_ASC.name,               operator='>', val2=1, lookBack=self.lookBack)
        self.v_PB_CoC_ByCountOpBars = sig.Validate(val1=self.PB_CoC_ByCountOpBars.name, operator='>', val2=1, lookBack=self.lookBack)
        self.v_PB_Overlap           = sig.Validate(val1=self.PB_Overlap.name,           operator='>', val2=1, lookBack=self.lookBack)
        self.l_pullback_vads = [self.v_PB_PctHLLH, self.v_PB_ASC, self.v_PB_CoC_ByCountOpBars, self.v_PB_Overlap]
        self.s_pullback_passed = sig.Score(name='s_pullback', sigs=self.l_pullback_vads, scoreType='mean', geThreshold=50, lookBack=self.lookBack)
        self.s_pullback_failed = sig.Score(name='s_pullback', sigs=self.l_pullback_vads, scoreType='mean', leThreshold=0, lookBack=self.lookBack)    

        # Validate Touches
        self.v_touchSupRes      = sig.Validate(val1=self.touchSupRes.name,      operator='>', val2=1, lookBack=self.lookBack)
        self.v_touchSupRes1Hour = sig.Validate(val1=self.touchSupRes1Hour.name, operator='>', val2=1, lookBack=self.lookBack)
        self.v_touchSupRes1Day  = sig.Validate(val1=self.touchSupRes1Day.name,  operator='>', val2=1, lookBack=self.lookBack)
        self.v_touchPrevDayLo   = sig.Validate(val1=self.touchPrevDayLo.name,   operator='>', val2=1, lookBack=self.lookBack)
        self.v_touchPrevDayHi   = sig.Validate(val1=self.touchPrevDayHi.name,   operator='>', val2=1, lookBack=self.lookBack)
        self.v_touchIntyra935Hi = sig.Validate(val1=self.touchIntyra935Hi.name, operator='>', val2=1, lookBack=self.lookBack)
        self.v_touchIntyra935Lo = sig.Validate(val1=self.touchIntyra935Lo.name, operator='>', val2=1, lookBack=self.lookBack)
        self.l_touches_vads = [self.v_touchSupRes, self.v_touchSupRes1Hour, self.v_touchSupRes1Day, self.v_touchPrevDayLo, self.v_touchPrevDayHi, self.v_touchIntyra935Hi, self.v_touchIntyra935Lo]
        self.s_touches_passed = sig.Score(name='s_touches', sigs=self.l_touches_vads, scoreType='mean', geThreshold=50, lookBack=self.lookBack)

        #! PB signals
        self.llx2 = sig.Lower(col='high', allLower=True, span=2, lookBack=self.lookBack)
        self.lhx2 = sig.Lower(col='low',  allLower=True, span=2, lookBack=self.lookBack)
        self.hhx2 = sig.Higher(col='high', allHigher=True, span=2, lookBack=self.lookBack)
        self.hlx2 = sig.Higher(col='low',  allHigher=True, span=2, lookBack=self.lookBack)

        #! Bounce Signals 
        self.rt_day_lo = sig.Retest(ls=self.ls, atrCol=self.ATR.name, direction=direction, valCol='day_low', withinAtrRange=self.retest_atrRange, rollingLen=self.retest_rollingLen, lookBack=self.lookBack, normRange=self.retest_normRange)
        self.colur_withLSx2 = sig.ColourWithLS(ls=self.ls, span=2, lookBack=self.lookBack)
        self.l_micro = [self.llx2, self.lhx2, self.hhx2, self.hlx2, self.rt_day_lo, self.colur_withLSx2]  #! add barSW, 
        self.s_micro_passed = sig.Score(sigs=self.l_micro, scoreType='cumsum', geThreshold=2, lookBack=self.lookBack)
        self.s_micro_failed = sig.Score(sigs=self.l_micro, scoreType='cumsum', leThreshold=0, lookBack=self.lookBack)



        # resets if new HP is formed
        strat = sig.Strategy('PB', lookBack=self.lookBack, ls=self.ls, riskAmount=100.00)

        """
        -- premarket higher volume than average
        -- time > 9:35 (wait for first 5 mins to play out)
        -- breaks premkt high
        -- price moves above first 5 min bar high
        -- price pulls back 
            a. to max 50% of the first 5 min bar
            b. 2 or more lower highs (LH)
            c. sequential pullback with less than 50% overlap on any bar
            d. touches a support level (prev day high, this day low, daily Res 1 lower )
        -- bullish bar completes (BSW ..  bot tail or CoC)
        -- buy signal is confirmed (RTM, RS, break prev bar high)
        """

        # validates various metris. each step must be fully validated before moving to the next step


        # prerequisites
        strat.pass_if(step=1, score=self.s_past_935)

        # price breaks
        if self.ls == 'LONG':
            strat.pass_if(step=2, score=self.s_above_5minPreMkt)
            strat.reset_if(step=2, score=self.s_below_5minPreMkt, startFromStep=2, cancelOrder=True)
        else:
            strat.pass_if(step=2, score=self.s_below_5minPreMkt)
            strat.reset_if(step=2, score=self.s_above_5minPreMkt, startFromStep=2, cancelOrder=True)

        # is trending
        strat.pass_if(step=3, score=self.s_trends_passed)
        strat.reset_if(step=3, score=self.s_trends_failed, startFromStep=2, cancelOrder=True)  #! TODO - currently not correct.... needs to reset if not trending

        # pullback
        strat.pass_if(step=3, score=self.s_pullback_passed)

        # touch
        strat.pass_if(step=4, score=self.s_touches_passed)


        # buillsh sigs
        strat.pass_if(step=4, countSigs=[], minCount=2)
        strat.reset_if(step=4, countSigs=[], maxCount=1)

        # below only if all steps are passed
        strat.set_prices(entry='high', stop='low', target='high')
        strat.reset_all_if(anySigs=[self.x, self.y, self.z], startFromStep=2, cancelOrder=True)
        strat.reset_all_if(allSigs=[self.x, self.y, self.z], startFromStep=2, cancelOrder=True)
        strat.reset_all_if(countSigs=[self.x, self.y, self.z], minCount=2, startFromStep=2, cancelOrder=True)
    


tap_1D = TAPresets1D()

tap_1D.l_sigs

In [ ]:
def normalize_value(val, norm_range=(0, 10)):

    min_val, max_val = norm_range
    
    # Handle zero value
    if val == 0:
        return 0
    
    # Handle positive values
    if val > 0:
        # Adjust the range to use only the positive portion
        effective_min = max(0, min_val)
        effective_max = max_val
        
        # Handle values below the minimum
        if val < effective_min:
            return 0
        
        # Handle values above the maximum
        if val > effective_max:
            return 100
        
        # Calculate the normalized value as a percentage
        normalized = ((val - effective_min) / (effective_max - effective_min)) * 100
        
    # Handle negative values
    else:  # val < 0
        # Adjust the range to use only the negative portion
        effective_min = min_val
        effective_max = min(100, max_val)
        
        # Handle values below the minimum
        if val < effective_min:
            return -100
        
        # Handle values above the maximum
        if val > effective_max:
            return 0
        
        # Calculate the normalized value as a percentage (negative)
        normalized = ((val - effective_min) / (effective_max - effective_min)) * -100
    
    return normalized

import pandas as pd
import numpy as np

# Using the updated normalize_value function defined above

# Create test data
test_data = {
    'value': [-15, -10, -5, -2, 0, 2, 5, 8, 10, 15, 20],
    'category': ['A', 'B', 'A', 'B', 'A', 'B', 'A', 'B', 'A', 'B', 'A']
}

# Create a DataFrame
df = pd.DataFrame(test_data)

# Test with different normalization ranges
print("Testing different normalization ranges:\n")

# Test case 1: Standard range (0, 10)
df['norm_0_10'] = df['value'].apply(lambda x: normalize_value(x, (0, 10)))

# Test case 2: Negative to positive range (-10, 10)
df['norm_neg10_10'] = df['value'].apply(lambda x: normalize_value(x, (-10, 10)))

# Test case 3: Narrow range (0, 5)
df['norm_0_5'] = df['value'].apply(lambda x: normalize_value(x, (0, 5)))

# Test case 4: All negative range (-20, -5)
df['norm_neg20_neg5'] = df['value'].apply(lambda x: normalize_value(x, (-20, -5)))

# Display results
print(df)

# Test specific examples from the original request
print("\n\nTesting specific examples:\n")
examples = [
    {'val': 5, 'norm': (0, 10)},      # Should be 50
    {'val': 5, 'norm': (0, 5)},       # Should be 100
    {'val': -5, 'norm': (0, 10)},     # Should be 0
    {'val': -5, 'norm': (-10, 10)},   # Should be -50
    {'val': 5, 'norm': (-10, 10)},    # Should be 50
    {'val': -5, 'norm': (-10, 0)}     # Should be -50
]

for ex in examples:
    result = normalize_value(ex['val'], ex['norm'])
    print(f"val = {ex['val']}, norm={ex['norm']} --> {result}")

# Additional examples to demonstrate edge cases
print("\nEdge cases:\n")
edge_cases = [
    {'val': 0, 'norm': (0, 10)},       # Zero
    {'val': 0, 'norm': (-10, 10)},     # Zero in mixed range
    {'val': 10, 'norm': (0, 10)},      # Maximum value
    {'val': -10, 'norm': (-10, 10)},   # Minimum in negative range
    {'val': 20, 'norm': (0, 10)},      # Value above maximum
    {'val': -15, 'norm': (-10, 10)}    # Value below minimum
]

for case in edge_cases:
    result = normalize_value(case['val'], case['norm'])
    print(f"val = {case['val']}, norm={case['norm']} --> {result}")

In [ ]:

val = 5
norm = (-10, 10) # Should be 50
vmin = norm[0]
vmax = norm[1]


def norm_pos(val, norm_range=(0, 10)):
    r_min, r_max = norm_range
    if val <= 0:
        return 0
    
    if val > 0:
        r_min = max(0, r_min)    
        if val > r_max:
            return 100
        return ((val - r_min) / (r_max - r_min)) * 100


def norm_neg(val, norm_range=(-10, 0)):
    r_min, r_max = norm_range
    if val == 0:
        return 0
    
    if val < 0:
        r_max = min(0, r_max)
        if val < r_min:
            return -100
        return ((val - r_max) / (r_min - r_max)) * -100
        

def norm_val_combined(val, norm_range=(-10, 10)):

    r_min, r_max = norm_range

    if r_min >= r_max:
        raise ValueError("norm_range must have (n1 < n2) values")
    
    if val == 0:
        return 0
    
    # Negative value - use the negative function if appropriate
    if val < 0 and r_min < 0:
        if r_min < 0:  # Range includes negative values
            return norm_neg(val, (r_min, min(0, r_max)))
        else:  # Range doesn't include negatives
            return -100
        
    # Positive value - use the positive function if appropriate
    else:
        if r_min >= 0:  # Range is entirely positive or starts at zero
            return norm_pos(val, norm_range)
        else:  # Range crosses zero, so use positive portion only
            return norm_pos(val, norm_range)
    

norm_pos(val=15, norm_range=(10,30))
norm_pos(val=-2, norm_range=(-20,-5))
norm_neg(-2, (-8, 0))

norm_val_combined(val=-15 , norm_range=(-20, -10))